In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
DATA_FILENAME = "csdh_data.csv"
DATA_FILEPATH = "/Users/callum/Uni/GitHubRepos/surviving-the-icu/datasets/drain_data/" + DATA_FILENAME

In [12]:
csdh = pd.read_csv(DATA_FILEPATH).iloc[:,1:] # drop first index column

In [13]:
csdh

,age,sex,hospital,diabetes,dementia,copd,stroke,ihd,arrhythmia,epilepsy,...,optype,surgeon,anaesthetic,csdhcolour,drain,drain_end,drain_entry,drain_days,timetoop_corrected,recurrence
0,20,1,17,0,0,0,0,0,0,1,...,1.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,15.0,1.0
1,23,1,1,0,0,0,0,0,0,0,...,1.0,3.0,1.0,5.0,1.0,1.0,1.0,2.0,15.0,0.0
2,23,0,26,0,0,0,0,0,0,0,...,1.0,3.0,1.0,1.0,1.0,1.0,NaN,2.0,5.0,0.0
3,23,1,17,0,0,0,0,0,0,0,...,1.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,10.0,0.0
4,25,0,10,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,96,0,18,0,0,0,0,0,0,0,...,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,0.0
789,96,0,3,0,1,0,0,0,0,0,...,1.0,2.0,2.0,3.0,1.0,1.0,NaN,1.0,5.0,0.0
790,97,1,4,0,0,0,0,0,0,0,...,1.0,3.0,2.0,1.0,1.0,1.0,NaN,1.0,1.0,0.0
791,98,1,18,0,0,0,0,0,0,0,...,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,0.0
